# Step3

任意の音源にモデルを適応させてみるフェーズ

In [26]:
model_path = 'model_layer5_20201219_163036.pt'
audio_path = "D:/yamamoto/test/test3.wav"

import torch
import torch.utils.data as utils
from librosa.core import istft,load,stft
# from librosa.output import write_wav

from IPython.display import display,Audio
from tqdm.notebook import tqdm
import glob
import numpy as np
import os
import random
import re

import model as mm
import utils as ut
from parameter import Parameter


p=Parameter()
datasets_save_dir = p.datasets_path
split = p.datasets_split #test/val/train
batch_size = p.batch_size
sample_rate = p.sample_rate
num_layer = p.num_layer
model_dir_path = p.model_path

clean_speech_dir = p.target_path
noise_dir = p.noise_path

audio_len = p.audio_len

fft_size = p.fft_size
hop_length = p.hop_length

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA is available:", torch.cuda.is_available())

c_data, sr_c = load(audio_path, sr=None)

if sr_c != sample_rate:
    c_data, _ = load(audio_path, sr=sample_rate)

if len(c_data) < audio_len:
    print("音声データが短すぎます。")

else:
    #modelのimport
    c_p = c_data[:audio_len]
    c_p_stft=stft(c_p, n_fft=fft_size, hop_length=hop_length)
    f = c_p_stft.shape[0]
    t = c_p_stft.shape[1]
    num_layer = int(re.sub("\\D", "", model_path)[0])
    model = mm.Net(t,f, num_layer)
    model.load_state_dict(torch.load(model_dir_path+model_path))

    # processing
    step = len(c_data) // audio_len

    for i in tqdm(range(step+1),leave=False,desc='[AUDIO Process..]'):

        if i == step:
            c_p = np.zeros(audio_len)
            c = c_data[i*audio_len :]
            c_p[:len(c)] = c

        else:
            c_p = c_data[i*audio_len : (i+1)*audio_len]


        addnoise_stft=stft(c_p, n_fft=fft_size, hop_length=hop_length)

        # modelに通す
        addnoise_tensor=np.abs(addnoise_stft).astype(np.float32)
        addnoise_tensor=torch.from_numpy(addnoise_tensor.astype(np.float32)).clone()
        
        ret = torch.zeros([batch_size,f,t])
        for r in range(batch_size):
            ret[r,:,:] = addnoise_tensor
            
        model.eval()
        mask = model(ret.float())[int(batch_size / 3),:,:]
        mask = mask.to('cpu').detach().numpy().copy()

        audio = addnoise_stft * mask

        audio =istft(audio, hop_length=hop_length)

        if i ==0:
            output = audio

        else:
            output = np.concatenate([output,audio])

    display(Audio(c_data,rate = sample_rate))
    display(Audio(output,rate = sample_rate))

CUDA is available: True
